# Tutorial: Hydrogen Electro Oxidation Reaction

## Introduction
 The microkinetic model of hydrogen electro-oxidation (HOR) proposed by Wang et al. [@Wang2007_Hydrogen] is implemented here. The corresponding input files are available in the software repository's example folder.




In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

from melektrodica import Collector, Calculator, Coordinator, Fitter

In [ ]:
directory = os.path.join("examples","Wang2007Hydrogen")